In [1]:
import os
import sys

from typing import List, Tuple
from pandas import read_csv
import pandas as pd
import random
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput
from bokeh.plotting import figure as bokeh_figure
from os.path import dirname, join
from bokeh.models import HelpTool, HoverTool
from bokeh.palettes import Category20c
from bokeh.transform import factor_cmap
from bokeh.transform import cumsum
from bokeh.models import FactorRange
from bokeh.transform import dodge
from bokeh.models import Panel, Tabs
from bokeh.layouts import column, row
from bokeh.models import Dropdown
from bokeh.palettes import RdYlBu3
from bokeh.palettes import Category20c

from bokeh.plotting import curdoc
from bokeh.models import ColumnDataSource, Div, Select
from bokeh.plotting import figure as bokeh_figure
from bokeh.models import CheckboxButtonGroup, CustomJS
from bokeh.models import MultiChoice
from bokeh.models import Paragraph
from masci_tools.vis.bokeh_plots import bokeh_scatter, bokeh_multi_scatter, bokeh_line
sys.path.append(".")


In [2]:
from analysis import filter_dataframe, calculate_crosstab
#from analysis import prepare_data_research_field
output_notebook()

Loading BokehJS ...

In [3]:
pwd = os.getcwd()
datafilepath_raw = join(pwd, './data/HMC_community_survey2021_merged_all_replies.csv')
datafilepath= join(pwd, './data/20211130_HMCCommSurvey_clean.csv')
surveydata = read_csv(datafilepath)

In [4]:
q1_key = 'careerLevel'
q2_key = 'docStructured'
df1 = calculate_crosstab(surveydata, q1_key, q2_key) 
df1

,careerLevel,docStructured,value,total,percentage
0,Director (of the institute),I am not sure,1,22,4.545455
1,Other,I am not sure,2,65,3.076923
2,PhD student,I am not sure,20,130,15.384615
3,Postdoc,I am not sure,31,128,24.218750
4,Principal Investigator,I am not sure,19,148,12.837838
5,Research associate,I am not sure,27,272,9.926471
6,Undergraduate / Masters student,I am not sure,1,24,4.166667
7,Director (of the institute),No,6,22,27.272727
8,Other,No,13,65,20.000000
9,PhD student,No,32,130,24.615385


In [5]:
list(df1['careerLevel'].value_counts().keys())

['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']

In [15]:
all_values = {}
for key in surveydata.keys():
    all_values[key] = list(surveydata[key].value_counts().keys())

In [17]:
all_values

{'Unnamed: 0.2': [0,
  678,
  665,
  666,
  667,
  668,
  669,
  670,
  671,
  672,
  673,
  674,
  675,
  676,
  677,
  679,
  663,
  680,
  681,
  682,
  683,
  684,
  685,
  686,
  687,
  688,
  689,
  690,
  691,
  692,
  664,
  662,
  505,
  646,
  633,
  634,
  635,
  636,
  637,
  638,
  639,
  640,
  641,
  642,
  643,
  644,
  645,
  647,
  661,
  648,
  649,
  650,
  651,
  652,
  653,
  654,
  655,
  656,
  657,
  658,
  659,
  660,
  693,
  694,
  695,
  741,
  728,
  729,
  730,
  731,
  732,
  733,
  734,
  735,
  736,
  737,
  738,
  739,
  740,
  742,
  696,
  743,
  744,
  745,
  746,
  747,
  748,
  749,
  750,
  751,
  752,
  753,
  754,
  755,
  727,
  726,
  725,
  724,
  697,
  698,
  699,
  700,
  701,
  702,
  703,
  704,
  705,
  706,
  707,
  708,
  709,
  710,
  711,
  712,
  713,
  714,
  715,
  716,
  717,
  718,
  719,
  720,
  721,
  722,
  723,
  632,
  631,
  630,
  551,
  538,
  539,
  540,
  541,
  542,
  543,
  544,
  545,
  546,
  547,
  548,
  549,

In [134]:
import math
df1['markersize'] = [2*math.sqrt(val/math.pi) for val in df1['percentage']]

In [135]:
selected = ColumnDataSource(df1)

In [7]:
df1.describe()

,value,total,percentage
count,21.000000,21.000000,21.000000
mean,24.333333,112.714286,20.461931
std,22.423946,82.636640,10.751577
min,1.000000,22.000000,3.076923
25%,6.000000,24.000000,13.636364
50%,20.000000,128.000000,20.000000
75%,32.000000,148.000000,26.351351
max,90.000000,272.000000,41.666667


In [136]:
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter)
from bokeh.transform import transform
from bokeh.models import Legend, Circle, LegendItem

default_tooltips = [("careerLevel", "@careerLevel"), 
                    ("docStructured", "@docStructured"),
                  ("total", "@total"),
                  ("percentage", "@percentage")]

def bokeh_corr_plot(source, x='value', y='counts',  figure=None, title='', x_range=None, y_range=None, 
                    markersize='markersize',  xlabel='Answers', ylabel='Number of answers', 
                    alpha=None, tooltips=default_tooltips, fill_color='#A0235A', **kwargs):
    """Plot an interactive circle with bokeh"""
    

    fig = bokeh_figure(height=600, width=900,
               title='title',
               toolbar_location='above',
               tools='hover,wheel_zoom,box_zoom,undo,reset,save',
                  y_range=y_range, x_range=x_range, tooltips=tooltips)

    circle = fig.circle(source=selected, x='careerLevel', y='docStructured', size='percentage', alpha=0.6, 
                        hover_alpha=1.0, line_color=fill_color, fill_color=fill_color, 
           hover_line_color='black', hover_line_width=5)#radius=0.1,
    fig.xaxis.major_label_orientation = 1
    fig.xaxis.axis_line_alpha= 0
    fig.yaxis.axis_line_alpha= 0
    
    #legend = Legend(
    #    title="Rel. amount\nof answers [%]",
    #    items=leg_items,
    #    location='top', orientation='vertical',
    #    border_line_color=None,
    #    title_text_font_size = '14px',
    #    title_text_font_style = 'bold'
    #)
    

    return fig
"""
p.circle([-40, -40], [-370, -390], color=list(gram_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["Gram-" + gr for gr in gram_color.keys()],
       text_font_size="9px", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
       color=list(drug_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(drug_color),
       text_font_size="12px", text_align="left", text_baseline="middle")
"""

def create_legend_items(number, size_min, color, fig):
    leg_items = []
    step = int(100/(number-1))
    size = size_min
    alpha_min = 0.2
    offset = 5
    y = -50
    y_step = int((600+2*y)/number)
    for i in range(number):
        
        circ = fig.circle(size=0.1+size*0.5, x=2.0, y=y, fill_color=color[i], name='foo')
        txt = fig.text(text=[f'{size}'], x=2.0+offset, y=y, text_font_size="22px", 
                       text_align="left", text_baseline="middle")
        y = y - y_step
        size = size + step
        leg_items.append(circ)
        leg_items.append(txt)
    return leg_items

def create_legend(fig, colors = ['red', 'blue', 'green', 'red', 'red', 'red']
):
    fig2 = bokeh_figure(height=600, width=200,
               title="Rel. amount\nof answers [%]",
               toolbar_location=None,
               tools='')
    leg_items = create_legend_items(5, size_min=0, color=colors, fig=fig2) #[("circle", [circle])]

    fig2.xaxis.axis_line_alpha= 0
    fig2.yaxis.axis_line_alpha= 0
    return fig2

In [137]:
y_range= ['No', 'I am not sure', 'Yes']
x_range= ['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']
fig = bokeh_corr_plot(source=selected, x='careerLevel', y='docStructured', markersize='markersize', 
                      x_range=x_range, y_range=y_range)
show(fig)

In [124]:
fig2 = create_legend(fig)
show(fig2)

In [126]:
fig2.xgrid.grid_line_alpha = 0
fig2.ygrid.grid_line_alpha = 0
fig2.xaxis.axis_label_text_alpha = 0
fig2.border_fill_alpha = 0
fig2.xaxis.major_label_text_alpha = 0
fig2.yaxis.major_label_text_alpha = 0
fig2.x_range.start = 0.0
fig2.x_range.end = 200
fig2.y_range.start = -600.0
fig2.y_range.end = 0.0
fig2.xaxis.minor_tick_line_alpha = 0
fig2.xaxis.major_tick_line_alpha = 0
fig2.yaxis.minor_tick_line_alpha = 0
fig2.yaxis.major_tick_line_alpha = 0
show(fig2)

In [127]:
fig.add_layout(fig2, 'right')

ValueError: failed to validate figure(id='20183', ...).right: expected an element of List(Instance(Renderer)), got seq with invalid items [figure(id='21113', ...)]

In [129]:
from bokeh.client import logging

In [130]:
logging?

In [112]:
a = fig.legend.items[3].renderers[0]

AttributeError: Trying to access 'items' attribute on an empty 'splattable' list

In [71]:
def hmc_barchart_style(fig: bokeh.models.figure, figure_kwargs: dict={}) -> bokeh.models.figure:
    """
    Define some figure properties for HMC dashboard barcharts
    """
    #def apply_property(fig, pro_val, pro_key):
    #    fig.

    fig.x_range.range_padding = 0.1
    fig.xgrid.grid_line_color = None
    fig.legend.location = "top_left"
    fig.legend.orientation = "horizontal"
    fig.y_range.start = 0
    fig.xaxis.major_label_orientation = 1
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    fig.title.text_font_size='25px'
    fig.yaxis.axis_label_text_font_size = '22px'
    fig.xaxis.axis_label_text_font_size = '22px'
    fig.xaxis.major_label_text_font_size = '18px'
    fig.xaxis.major_label_text_font_size = '18px'
    fig.toolbar.logo = None
    fig.legend.location = "top_right"
    fig.legend.orientation = "vertical"
    fig.legend.click_policy="hide"
    }
    


    return figure_kwargs


SyntaxError: unmatched '}' (3966699803.py, line 25)

In [ ]:
bokeh_figure?

In [ ]:
fig.__setattr__?

In [ ]:
from bokeh.core.json_encoder import serialize_json

In [ ]:
fig.properties()

In [ ]:
fig.set_from_json?

In [ ]:
fig.axis

In [ ]:
fig['axis']

In [ ]:
fig.properties('axis')

In [ ]:
fig.

In [ ]:
def apply_property(fig, pro_key, pro_val):
    if '.' in pro_key:
        pro_key_new = pro_key.split('.')[]
        apply_property(un_object, pro_key_new, pro_val)
    else:
        fig.set_from_json(pro_key, pro_val)
    return fig

In [ ]:
fig =apply_property(fig, 'title', "top_left")

In [ ]:
show(fig)

In [ ]:

y_range= ['No', 'I am not sure', 'Yes']
x_range= ['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']
fig = bokeh_figure(height=600, width=900,
               title='title',
               toolbar_location='above',
               tools='hover,wheel_zoom,box_zoom,undo,reset,save',
                  y_range=y_range, x_range=x_range)

circle = fig.circle(source=selected, x='careerLevel', y='docStructured', size='percentage', alpha=0.6, hover_alpha=1.0, 
           hover_line_color='black', hover_line_width=5)#radius=0.1,
fig.xaxis.major_label_orientation = 1
fig.xaxis.axis_line_alpha= 0
fig.yaxis.axis_line_alpha= 0

def create_legend_items(number, size_min, color):
    leg_items = []
    step = int(100/(number-1))
    size = size_min
    alpha_min = 0.2
    for i in range(number):
        circ = fig.circle(radius=size, fill_color=color[i], name='foo')
        leg_items.append((f'{size}', [circ]))
        size = size + step
    
    return leg_items

colors = ['red', 'blue', 'green', 'red', 'red', 'red']
leg_items = create_legend_items(5, size_min=0, color=colors) #[("circle", [circle])]
legend = Legend(
        title="Rel. amount\nof answers [%]",
        items=leg_items,
        location='top', orientation='vertical',
        border_line_color=None,
        title_text_font_size = '14px',
        title_text_font_style = 'bold'
)
fig.add_layout(legend, 'right')
show(fig)

In [ ]:
a = legend.items[0]


In [ ]:
a

In [ ]:
legend.title

In [ ]:
fig = bokeh_corr_plot(source=selected, x='careerLevel', y='docStructured', markersize='percentage')
show(fig)